# Visualize Employees and their group membership

In [17]:
import pandas as pd

In [18]:
Users = pd.read_excel("data/data.xlsx", 'Users')
Users

,Username,Name,Title,Email,Group
0,BrandtN,"Brandt, Niklas",IT-Partner,niklas.brandt@skanska.se,36704 Regional IT Support
1,BjorklundMi,"Björklund, Mikael",Funktionschef,mikael.bjorklund@skanska.se,36701 Common Services HOF
2,BemingC,"Beming, Cecilia",Kategoriansvarig,cecilia.beming@skanska.se,31320 Inköp Indirekt
3,AnderssAsa,"Bergström, Åsa",Marknadskoordinator,asa.bergstrom@skanska.se,41800 SIS OpC Norr Stab
4,AhlqvistM,"Ahlqvist, Mikael",Gruppchef,mikael.ahlqvist@skanska.se,36703 Business Platforms
...,...,...,...,...,...
6688,HellgrenM,"Hellgren, Magnus",Bergarbetare,magnus.hellgren@skanska.se,41223 SIS Prod Losshållning Norr och Öst
6689,AronssonC,"Aronsson, Christer",Väg & Anläggningsarbetare,christer.aronsson@skanska.se,71660 VoA Norr DC Västernorrl/Jämtl Stab
6690,SetermoC,"Setermo, Carl",Träarbetare,carl.setermo@skanska.se,71440 VoA MSv DC Bergslagen Stab
6691,OcskoT,"Ocsko, Tobias",Projektingenjör,tobias.ocsko@skanska.se,52530 Hus Sthlm Syd Distrikt 3 Stab


In [19]:
groups = pd.read_excel("data/data.xlsx", 'Groups', dtype="string")
groups[['code','Display Name']] = groups["Name"].str.split(" ", n=1, expand=True)
groups.drop(["code"],axis=1, inplace=True)
groups

,Name,Boss Email,Boss Name,Display Name
0,36803 HR and Learning,pernilla.widlund@skanska.se,Pernilla Widlund,HR and Learning
1,36703 Business Platforms,mikael.ahlqvist@skanska.se,Mikael Ahlqvist,Business Platforms
2,36806 Customer facing services,malin.hormanseder@skanska.se,Malin Hörmanseder,Customer facing services
3,36605 Data Foundation,cecilia.backa@skanska.se,Cecilia Backa,Data Foundation
4,36706 Digital arbetsplats,kristoffer.winberg@skanska.se,Kristoffer Winberg,Digital arbetsplats
5,36705 Digital Collaboration,karam.chammas@skanska.se,Karam Chammas,Digital Collaboration
6,36802 Entreprenad,linn.areno@skanska.se,Linn Areno,Entreprenad
7,36804 Finance and Procurement,jessica.axelsson@skanska.se,Jessica Axelsson,Finance and Procurement
8,36702 IT Service Desk,goran.liljeberg@skanska.se,Göran Liljeberg,IT Service Desk
9,36704 Regional IT Support,joakim.vahatalo@skanska.se,Joakim Vähätalo,Regional IT Support


In [20]:
dataview = Users[Users['Group'].isin(groups["Name"])]
dataview = dataview.drop(['Username', 'Email','Title'], axis=1)
dataview

,Name,Group
0,"Brandt, Niklas",36704 Regional IT Support
4,"Ahlqvist, Mikael",36703 Business Platforms
11,"Forsberg, Jimmy",36802 Entreprenad
109,"Strömstedt, Ludwig",36804 Finance and Procurement
122,"Ljungberg, Linus",36804 Finance and Procurement
...,...,...
6654,"Espino, Marione",36703 Business Platforms
6660,"Santra, Snehashis",36804 Finance and Procurement
6669,"Stevall, Robert",36806 Customer facing services
6670,"Ramesh, Nireeksha",36806 Customer facing services


In [21]:
dataview["Group"].drop_duplicates().info()

<class 'pandas.core.series.Series'>
Index: 10 entries, 0 to 1827
Series name: Group
Non-Null Count  Dtype 
--------------  ----- 
10 non-null     object
dtypes: object(1)
memory usage: 160.0+ bytes


In [22]:
import networkx as nx
import numpy as np
from pyvis.network import Network

visualizationdata = pd.DataFrame(
    {
        "source": dataview["Name"].tolist(),
        "target": dataview["Group"].tolist(),
        "weight": 0
    }
)
G_team_members = nx.from_pandas_edgelist(visualizationdata, edge_attr=True)
net = Network(notebook=True)
for source in visualizationdata['source'].unique():
    net.add_node(source, label=source)
for target in visualizationdata['target'].unique():
    net.add_node(target, label=target, color='red')
for index, row in visualizationdata.iterrows():
    net.add_edge(row['source'], row['target'], value=row['weight']/3, color='gray')
net.show("employee_group.html")

with open("employee_group.html", "r") as file:
    html_data = file.read()


employee_group.html


In [23]:

start = html_data.find("nodes =")
end = html_data.find("nodeColors = {};")
javascript_data = html_data[start:end].strip()
javascript_data = javascript_data.replace("nodes = ", "group_nodes = ")
javascript_data = javascript_data.replace("edges = ", "group_edges = ")
print(javascript_data)
file = open("data/employee_group.js", "wt") #(r)ead, (a)ppend, (w)rite, create(x) (t)ext/(b)inary
file.write(javascript_data)
file.close()

import os
os.remove("employee_group.html")

group_nodes = new vis.DataSet([{"color": "#97c2fc", "id": "Brandt, Niklas", "label": "Brandt, Niklas", "shape": "dot"}, {"color": "#97c2fc", "id": "Ahlqvist, Mikael", "label": "Ahlqvist, Mikael", "shape": "dot"}, {"color": "#97c2fc", "id": "Forsberg, Jimmy", "label": "Forsberg, Jimmy", "shape": "dot"}, {"color": "#97c2fc", "id": "Str\u00f6mstedt, Ludwig", "label": "Str\u00f6mstedt, Ludwig", "shape": "dot"}, {"color": "#97c2fc", "id": "Ljungberg, Linus", "label": "Ljungberg, Linus", "shape": "dot"}, {"color": "#97c2fc", "id": "Haglund, Marie", "label": "Haglund, Marie", "shape": "dot"}, {"color": "#97c2fc", "id": "Brattl\u00f6w, Jonna", "label": "Brattl\u00f6w, Jonna", "shape": "dot"}, {"color": "#97c2fc", "id": "Widlund, Pernilla", "label": "Widlund, Pernilla", "shape": "dot"}, {"color": "#97c2fc", "id": "Jansson, Torbj\u00f6rn", "label": "Jansson, Torbj\u00f6rn", "shape": "dot"}, {"color": "#97c2fc", "id": "Ljungberg, Henrik", "label": "Ljungberg, Henrik", "shape": "dot"}, {"color": "